# Reps Report

This notebook generates an emg and kinematics report from repetitive movements segemented by labels.
Protocols with repeat labesl can use this report to look at emg, limb position, and joint angles for each labeled segment

This report can be parameterized by defining a movements dictionary in the runner

```
{ 
  "movements" : {
    "r_raise" : {
        "label"  : "raise",
        "emgs"   : ["r_shank_emg"],
        "limbs" : ["r_shank","r_foot"],
        "joints" : [
             {
                "a" : "r_foot", 
                "b" : "r_shank",
                "-x" : "dorsi_flexion",
                "-y" : "ankle_eversion",
                "-z" : "ankle_rotation"
             }
        ]
    }
  }
}
```

For more fun try parameterizing the presentation as well

```
{
  "presentation" : {
    "style"  : "seaborn-pastel",
    "title"  : "off",
    "legend" : "off"
}
```

In [ ]:
%load_ext autoreload
%autoreload 2

import os

import numpy as np
import pandas as pd

import cionic
from cionic import tools
from cionic.kinematics import CycleAxis, Kinematics

## Load Collection

Download or construct the npz for the collection to be analyzed, and save it to the recordings directory.

As a sanity check print out the **positions**, **labels**, and **stream names**


In [ ]:
# param

npzpath = None
download = None
tokenpath = None
presentation = {}

In [ ]:
if download:
    cionic.auth(tokenpath=tokenpath)
    cionic.download_npz(npzpath, download)

npz = cionic.load_segmented(npzpath)
regs = tools.stream_regs(np.load(npzpath))  # can only be loaded from original npz
outpath = os.path.splitext(npzpath)[0] + '.pdf'

if 'position' in npz['segments'].dtype.names:
    print(f"Positions: {set(npz['segments']['position'])}")

if 'label' in npz['segments'].dtype.names:
    print(f"Labels: {set(npz['segments']['label'])}")

if 'stream' in npz['segments'].dtype.names:
    print(f"Stream: {set(npz['segments']['stream'])}")

## Configure Movement Processor

Using the markup definition described at the beginning of this document, load the neccessary data to render each movement

In [ ]:
segments = pd.DataFrame(npz['segments'])

# initialize kinematics
k = Kinematics({})

# load streams that match label and positions
for movement in movements.keys():

    # calculate needed positions from emgs, angles, joints
    positions = set(
        movements[movement].get('emgs', []) + movements[movement].get('limbs', [])
    )
    for joint in movements[movement].get('joints', []):
        positions.add(joint["a"])
        positions.add(joint["b"])

    streams = segments.query(f'position in {list(positions)}')
    for index, seg in streams.iterrows():
        # to override elapsed_s with time calculated by sample rate
        # k.load_array(side, seg['position'], seg['stream'], npz[seg['path']], regs.get(seg['device'], hz=seg['avg_rate_hz'])

        # to select a specific time range out of a longer recording
        # k.load_array(side, seg['position'], seg['stream'], npz[seg['path']], regs.get(seg['device'], time_range=[170.0,220.0])

        k.load_array(
            movement,
            seg['position'],
            seg['stream'],
            npz[seg['path']],
            regs.get(seg['device']),
            time_range=[np.min(streams['start_s']), np.max(streams['end_s'])],
        )

        if 'chanpos' in seg:
            k.load_channel_pos(movement, seg['position'], seg['stream'], seg['chanpos'])

        if 'calibration' in seg:
            k.load_calibration(
                movement, seg['position'], seg['stream'], seg['calibration']
            )

## Calculate Normalized EMG

Compute normalized EMG according to **emg_params** 

normalization algorithms
* k.butter_emg 1. convert to uV with 4V reference 2. signal.butter 3. RMS 
* k.butter_emg 1. convert to uV with 4V reference 2. signal.firwin 3. RMS




In [ ]:
emg_filter_butter = {
    'filter': tools.butter_highpass_filter,
    'filter_order': 5,
    'cutoff_freq': 50,
    'rms_window': 301,
    'normalize': None,
}

for movement in movements.keys():
    k.calculate_emgs(movement, movements[movement].get('emgs', []), emg_filter_butter)

## Calculate Limb and Joint Angles

Once the kinematics engine is configured, we can compute the angles of the limbs and the angles of the joints.

This step applies the sensor calibrations and then converts the results into euler angles for presentation.

In [ ]:
# calculate joint angles an group
for movement in movements.keys():
    joints = movements[movement].get('joints', [])
    if joints:
        # convert jsonable dict to kinematics expected form
        jc = {
            (j["a"], j["b"]): {a: j[a] for a in j if a not in ["a", "b"]}
            for j in joints
        }
        k.calculate_joint_angles(movement, jc)
    limbs = movements[movement].get('limbs', [])
    if limbs:
        k.calculate_limb_angles(movement, limbs)

## Calculate Splits

Using study labels create a set of splits named *segment*

In [ ]:
for name, config in movements.items():
    label = config["label"]
    k.calculate_segment_splits(name, label, 'segment', streams)

## Generate Report

Visualize splits and save to pdf

In [ ]:
k.save_open(outpath)

for name, config in movements.items():
    # draw the emg graphs
    emgs = config.get("emgs")
    if emgs:
        emg_config = {name: {}}
        for emg in emgs:
            emg_config[name].update(
                {
                    emg: [
                        pos
                        for pos in k.groups[name][emg].keys()
                        if pos not in ["adcf", "emg"]
                    ]
                }
            )
        k.save_text(f"{name} emg", width=20, height=10)
        k.plot_splits(
            'segment',
            config=emg_config,
            component='emg',
            width=20,
            sheight=5,
            xaxis=CycleAxis(),
            markers={},
            presentation=presentation,
        )
    # draw the limbs
    limbs = config.get("limbs")
    if limbs:
        limb_config = {name: {}}
        for limb in limbs:
            # pull the limb angles up one level in the kinematics object
            # and then draw them
            components = [
                c for c in k.groups[name][limb]["euler"].dtype.names if c != "elapsed_s"
            ]
            for component in components:
                rekey = f"{limb}_{component}"
                k.groups[name][rekey]["angle"] = k.groups[name][limb]["euler"][
                    [component, "elapsed_s"]
                ]
                k.groups[name][rekey]["angle"].dtype.names = ["degrees", "elapsed_s"]
                limb_config[name].update({rekey: ["angle"]})
        k.save_text(f"{name} limbs", width=20, height=10)
        k.plot_splits(
            'segment',
            config=limb_config,
            component='degrees',
            width=20,
            sheight=5,
            xaxis=CycleAxis(),
            markers={},
            presentation=presentation,
        )
    # draw the joints
    joints = config.get("joints")
    if joints:
        joint_config = {name: {}}
        for joint in joints:
            joint_config[name].update(
                {n: ["angle"] for (a, n) in joint.items() if a not in ["a", "b"]}
            )
        k.save_text(f"{name} joints", width=20, height=10)
        k.plot_splits(
            'segment',
            config=joint_config,
            component='degrees',
            width=20,
            sheight=5,
            xaxis=CycleAxis(),
            markers={},
            presentation=presentation,
        )

k.save_close()